In order to load a file we use [fiona](https://github.com/Toblerity/Fiona)

In [1]:
import fiona
from shapely.wkt import loads
from shapely.geometry import shape
import numpy as np
from pprint import pprint

In [2]:
desired_footprint = loads(
    "POLYGON ((8.507258495787331 47.36630071672813, 8.507194818858338 47.36627707321725, 8.50718797218813 47.36628557397753, 8.507063333295594 47.366239297804114, 8.507063342324688 47.36623928659597, 8.50684791972349 47.36615928441324, 8.50663515187207 47.36608026462807, 8.506635147463841 47.36608027010266, 8.506510216160828 47.36603386254508, 8.506516197089956 47.366026422258926, 8.506448545473718 47.366001285902954, 8.506368130599874 47.36610123411666, 8.506563688811562 47.366173851455706, 8.50656369036007 47.366173849530526, 8.506988922194576 47.366331785799886, 8.506988924911536 47.36633178242341, 8.507177696213391 47.3664018924277, 8.507217715202716 47.366351789323986, 8.507258495787331 47.36630071672813))"
)

In [3]:
# Let's get the right filename
centroid = desired_footprint.centroid
simulation_type = "sun"
filename = (
    f"{simulation_type}_N{int(centroid.y * 100):05d}_E{int(centroid.x * 100):05d}.fgb"
)

In [4]:
points_of_interest = []
with fiona.open(filename) as data:
    data_filtered = data.filter(bbox=desired_footprint.bounds)
    for obs_point in data_filtered:
        if (
            shape(obs_point["geometry"]).within(desired_footprint)
            and obs_point["properties"]["level"] == 2
        ):
            points_of_interest.append(obs_point)

In [5]:
# Now we have the observations and the values. We can check some statistics
values = [p["properties"]["201806211600"] for p in points_of_interest]
stats = dict(
    p20=float(np.percentile(values, 20)),
    p80=float(np.percentile(values, 80)),
    median=float(np.median(values)),
    min=float(np.min(values)),
    max=float(np.max(values)),
    mean=float(np.mean(values)),
    stddev=float(np.std(values)),
    count=len(values),
)
pprint(stats)

{'count': 651,
 'max': 14.40117,
 'mean': 2.3873358678955454,
 'median': 1.41579,
 'min': 0.00417,
 'p20': 0.4235,
 'p80': 3.97665,
 'stddev': 2.5077285180785847}
